In [187]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
# Load the data
data_dir = 'C:/Users/Dell/Desktop'
df = pd.read_csv(data_dir + '/winequality-white 2022.csv', sep = '\;')
df = df[:800]
print(df.shape)
df.head()

(800, 12)


<ipython-input-187-2f63cc49f982>:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(data_dir + '/winequality-white 2022.csv', sep = '\;')


,"""fixed acidity","""""volatile acidity""""","""""citric acid""""","""""residual sugar""""","""""chlorides""""","""""free sulfur dioxide""""","""""total sulfur dioxide""""","""""density""""","""""pH""""","""""sulphates""""","""""alcohol""""","""""quality"""""""
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,5
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [209]:
# Construct column 'good wine'
df['good wine'] = df['""quality"""'].apply(lambda x: 1 if x>=6 else 0)
df['good wine'].describe()

count    800.000000
mean       0.166250
std        0.372538
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: good wine, dtype: float64

In [162]:
# Generate Test Set 
X_origin, X_test, Y_origin, Y_test = train_test_split(df.iloc[:, :12], df.iloc[:, -1], test_size=0.250, random_state=0)
print(X_origin.shape)
print(X_test.shape)

(600, 12)
(200, 12)


In [163]:
# Generate Training and Validation Sets 
X_train, X_valid, Y_train, Y_valid = train_test_split(X_origin, Y_origin, test_size=0.333, random_state=0)
print('Current Training set shape is:', X_train.shape)
print('Current Validation set shape is:', X_valid.shape)
print('Current Testing set shape is:', X_test.shape)

Current Training set shape is: (400, 12)
Current Validation set shape is: (200, 12)
Current Testing set shape is: (200, 12)


In [178]:
# Z-score normalisation for training set, validation set and test set data
scaler = StandardScaler()
X_train, X_valid, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_valid), scaler.fit_transform(X_test)
X_train, X_valid, X_test

(array([[ 2.06221733e-01,  6.05153255e-01, -2.15347587e-01, ...,
         -1.31224975e+00, -4.98380769e-01, -1.63777700e-01],
        [-1.10146205e+00,  4.69741920e-01, -8.96288572e-01, ...,
         -3.14640658e-01, -5.99216534e-01, -2.21099895e+00],
        [ 9.38524652e-01, -4.51055156e-01,  4.16954757e-01, ...,
         -2.62134916e-01, -5.99216534e-01, -1.63777700e-01],
        ...,
        [ 5.72373192e-01,  2.53083785e-01,  2.78456224e-02, ...,
         -2.09629174e-01,  1.06633822e-01, -1.63777700e-01],
        [-3.69159132e-01,  8.48893657e-01, -3.61263512e-01, ...,
          1.57911019e-01, -7.00052299e-01, -1.63777700e-01],
        [-3.00767270e-03, -1.37185223e+00,  2.22400190e-01, ...,
          3.62328998e+00,  2.07469587e-01,  1.88344355e+00]]),
 array([[-0.5601507 ,  0.06220031, -0.87017219, ..., -0.78254711,
         -0.7108312 , -0.1413925 ],
        [-0.50859518, -0.82836347, -0.05288339, ...,  0.99117003,
         -0.60088051, -0.1413925 ],
        [-0.66326173, -0.

In [194]:
# Load KNN with classifiers k = 1, 2,..., 100
# Find optimised classifier k with highest accuracy(i.e.metric selected: accuracy_score)
predict_dict = dict()
for k in range(1, 101):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, Y_train)
    prediction_valid = neigh.predict(X_valid)
    predict_dict[k] = accuracy_score(prediction_valid, Y_valid)
K_1st = max(predict_dict, key=predict_dict.get)
K_1st

3

In [196]:
# Test performance on Test set
# Find generalisation error
neigh_1st = KNeighborsClassifier(n_neighbors=K_1st)
neigh_1st.fit(X_train, Y_train)
predic_1st_test = neigh_1st.predict(X_test)
print(accuracy_score(predic_1st_test, Y_test))
print(classification_report(predic_1st_test, Y_test))
print(confusion_matrix(predic_1st_test, Y_test))

0.98
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       166
           1       0.92      0.97      0.94        34

    accuracy                           0.98       200
   macro avg       0.96      0.98      0.97       200
weighted avg       0.98      0.98      0.98       200

[[163   3]
 [  1  33]]


In [212]:
# Redo steps 4 to 7 under new splitting
X_origin, X_test, Y_origin, Y_test = train_test_split(df.iloc[:, :12], df.iloc[:, -1], test_size=0.500, random_state=0)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_origin, Y_origin, test_size=0.500, random_state=0)

scaler = StandardScaler()
X_train, X_valid, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_valid), scaler.fit_transform(X_test)

predict_dict = dict()
for k in range(1, 101):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, Y_train)
    prediction_valid = neigh.predict(X_valid)
    predict_dict[k] = accuracy_score(prediction_valid, Y_valid)
new_K_1st = max(predict_dict, key=predict_dict.get)
new_K_1st

new_neigh_1st = KNeighborsClassifier(n_neighbors=new_K_1st)
new_neigh_1st.fit(X_train, Y_train)
new_predic_1st_test = new_neigh_1st.predict(X_test)
print(accuracy_score(new_predic_1st_test, Y_test))
print(classification_report(new_predic_1st_test, Y_test))
print(confusion_matrix(new_predic_1st_test, Y_test))

0.97
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       337
           1       0.84      1.00      0.91        63

    accuracy                           0.97       400
   macro avg       0.92      0.98      0.95       400
weighted avg       0.97      0.97      0.97       400

[[325  12]
 [  0  63]]


In [ ]:
# Insight:  ? ? ?